In [0]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Bidirectional,TimeDistributed

In [0]:
df_train = pd.read_csv('../content/drive/My Drive/train.csv')
df_train

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0
...,...,...,...
4999995,499.9996,2.9194,7
4999996,499.9997,2.6980,7
4999997,499.9998,4.5164,8
4999998,499.9999,5.6397,9


In [0]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train.open_channels)
#signals = np.reshape(signal,(1))
data = np.zeros((len(signal), 2))
data[:, 0] = signal
data[:, 1] = open_channels
data = data.reshape((1000, 5000, 2))
np.random.shuffle(data)
x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1].reshape((1000,5000,1))

In [0]:
model = Sequential()
model.add(Bidirectional(LSTM(units=20, return_sequences=True, input_shape=(5000,1))))
# model.add(Dense(20,activation='relu'))
# model.add(Dense(units=1,activation='relu'))
model.add(TimeDistributed(Dense(units=1,activation='relu')))
model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])

In [0]:
# train LSTM
for epoch in range(1000):
	# fit model for one epoch on this sequence
  print( "n_batch: " + str(epoch))
  model.fit(x[epoch].reshape(1,5000,1), y[epoch].reshape(1,5000,1), epochs=1, batch_size=1, verbose=1)

n_batch: 0
1/1 [==============================] - 0s 1ms/step - loss: 0.1964 - accuracy: 0.0112
n_batch: 1
1/1 [==============================] - 0s 2ms/step - loss: 0.0223 - accuracy: 0.9776
n_batch: 2
1/1 [==============================] - 0s 1ms/step - loss: 0.7461 - accuracy: 0.1776
n_batch: 3
1/1 [==============================] - 0s 1ms/step - loss: 0.0322 - accuracy: 0.9594
n_batch: 4
1/1 [==============================] - 0s 1ms/step - loss: 0.4431 - accuracy: 0.3186
n_batch: 5
1/1 [==============================] - 0s 1ms/step - loss: 0.1313 - accuracy: 0.8956
n_batch: 6
1/1 [==============================] - 0s 1ms/step - loss: 7.4589 - accuracy: 0.0000e+00
n_batch: 7
1/1 [==============================] - 0s 1ms/step - loss: 3.0737 - accuracy: 4.0000e-04
n_batch: 8
1/1 [==============================] - 0s 1ms/step - loss: 0.2720 - accuracy: 0.1504
n_batch: 9
1/1 [==============================] - 0s 1ms/step - loss: 10.9881 - accuracy: 0.1256
n_batch: 10
1/1 [==============

In [0]:
predictions = model.predict(x)

In [0]:
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = round(y[i][j][0])
        pred = round(predictions[i][j][0])
        if actual == 0 and pred == 0:
            tn += 1
        elif actual == 0 and pred > 0:
            fp += 1
        elif actual > 0 and pred == 0:
            fn += 1
        else:
            tp += 1
total = tp+tn+fp+fn

In [0]:
print("Accuracy:",(tp+tn)*100/total)
print("True Positive Rate:",tp*100/total)
print("True Negative Rate:",tn*100/total)
print("False Postiive Rate:",fp*100/total)
print("False Negative Rate:",fn*100/total)

Accuracy: 94.81222
True Positive Rate: 74.38024
True Negative Rate: 20.43198
False Postiive Rate: 4.37106
False Negative Rate: 0.81672
